## Imports 

In [ ]:
import argparse
import easydict
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from matplotlib import pyplot as plt
from plotread import *
import shap
import pickle
from collections import Counter
%matplotlib inline

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# init the JS visualization code
shap.initjs()

In [ ]:
import tensorflow._api.v2.compat.v1 as tf                 # does not work for python=3.7
from tensorflow.compat.v1.keras.backend import get_session
tf.compat.v1.disable_v2_behavior()                          # does not work for thesis, except with DeepExplainer
tf.compat.v1.disable_eager_execution()                    # does not work

from tensorflow.python.ops.numpy_ops import np_config     # does not work for python=3.7
np_config.enable_numpy_behavior()                         # does not work for python=3.7

## Functions 

In [ ]:
def main():
    ###########################################################################
    # Parser Definition
    ###########################################################################
    opt = easydict.EasyDict({
        "model": "GRU",
        "datapath": "Dataset1/",
        "savepath": "Experiment1/GRU/8/Singleoutput/Runy/",
        "extension": ".dat",
        "batch_size": 32,
        "plots_in": False,
        "plots_out": True
    })

    ###########################################################################
    # Variables Definition
    ###########################################################################
    nin = ['time', 'PLML2', 'PLMR', 'AVBL', 'AVBR']
    nout = ['time', 'DB1', 'LUAL', 'PVR', 'VB1']
    neurons = ['time','DB1','LUAL','PVR','VB1','PLML2','PLMR','AVBL','AVBR']

    ###########################################################################
    # Read data
    ###########################################################################
    files = getdata(opt.datapath, opt.extension)
    train, valid, test = splitdata(files)
    trainx, trainy = readdata(opt.datapath, train, neurons, nin, nout)
    validx, validy = readdata(opt.datapath, valid, neurons, nin, nout)
    testx, testy = readdata(opt.datapath, test, neurons, nin, nout)
    if opt.plots_in:
        plotdata(trainx, '/train_data', '/x', opt.model, opt.savepath)
        plotdata(trainy, '/train_data', '/y', opt.model, opt.savepath)
        plotdata(validx, '/valid_data', '/x', opt.model, opt.savepath)
        plotdata(validy, '/valid_data', '/y', opt.model, opt.savepath)
        plotdata(testx, '/test_data', '/x', opt.model, opt.savepath)
        plotdata(testy, '/test_data', '/y', opt.model, opt.savepath)
        
    ###########################################################################
    # Load Model and Evaluate
    ###########################################################################
    output_size = 1
    global model
    model = load_model(opt.savepath + 'model.h5')
    model.summary()

    print("Starting to explain...")
    X_train = np.array(trainx)
    X_test = np.array(testx)

    nin_names = nin[1:len(nin)]

    return X_train, X_test, nin_names

## Code 

In [ ]:
X_train, X_test, nin_names = main()

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
background = X_train[np.random.choice(X_train.shape[0], 1, replace=False)]
print("background.shape=")
print(background.shape)

In [ ]:
background

In [ ]:
for layer in model.layers:
    print("LAYER")
    print(layer.name, layer.inbound_nodes, layer.outbound_nodes)
    print(layer.input_shape)
    print(layer.output_shape) 

In [ ]:
X_test[:1,:,:]

### explainers

In [ ]:
# explainer1 = shap.DeepExplainer(model, X_train)
# explainer2 = shap.DeepExplainer(model, background)

explainer3 = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), X_train)
explainer4 = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), background)

# explainer5 = shap.GradientExplainer(model, X_train)
# explainer6 = shap.GradientExplainer(model, background)

# explainer7 = shap.GradientExplainer((model.layers[0].input, model.layers[-1].output), X_train)
# explainer8 = shap.GradientExplainer((model.layers[0].input, model.layers[-1].output), background)

### shap_values

#### Loading the shap_values from pickle files

In [ ]:
# with open("D:\Pedro\IST\ANO 2\SEM 2\Tese\gmestre\ICLR-RNN-CElegans\Experiment1\GRU\8\Singleoutput\Runy\pickles\shap_values1_explainer4.pkl", 'rb') as file:
#     shap_values1_explainer4 = pickle.load(file)

In [ ]:
# list_of_shap_values = []

# for i in range(1, 9):
#     for j in range(1, 3):
#         with open("D:\Pedro\IST\ANO 2\SEM 2\Tese\gmestre\ICLR-RNN-CElegans\Experiment1\GRU\8\Singleoutput\Runy\pickles\shap_values{}_explainer{}.pkl".format(j, i), 'rb') as file:
#             shap_value = pickle.load(file)
#             list_of_shap_values.append(shap_value)

In [ ]:
# shap_values1_explainer1 = list_of_shap_values[0]
# shap_values2_explainer1 = list_of_shap_values[1]
# shap_values1_explainer2 = list_of_shap_values[2]
# shap_values2_explainer2 = list_of_shap_values[3]
# shap_values1_explainer3 = list_of_shap_values[4]
# shap_values2_explainer3 = list_of_shap_values[5]
# shap_values1_explainer4 = list_of_shap_values[6]
# shap_values2_explainer4 = list_of_shap_values[7]
# shap_values1_explainer5 = list_of_shap_values[8]
# shap_values2_explainer5 = list_of_shap_values[9]
# shap_values1_explainer6 = list_of_shap_values[10]
# shap_values2_explainer6 = list_of_shap_values[11]
# shap_values1_explainer7 = list_of_shap_values[12]
# shap_values2_explainer7 = list_of_shap_values[13]
# shap_values1_explainer8 = list_of_shap_values[14]
# shap_values2_explainer8 = list_of_shap_values[15]

In [ ]:
list_of_shap_values = []

for i in range(3, 5):
    for j in range(1, 3):
        with open("D:\Pedro\IST\ANO 2\SEM 2\Tese\gmestre\ICLR-RNN-CElegans\Experiment1\GRU\8\Singleoutput\Runy\pickles\shap_values{}_explainer{}.pkl".format(j, i), 'rb') as file:
            shap_value = pickle.load(file)
            list_of_shap_values.append(shap_value)

In [ ]:
shap_values1_explainer3 = list_of_shap_values[0]
shap_values2_explainer3 = list_of_shap_values[1]
shap_values1_explainer4 = list_of_shap_values[2]
shap_values2_explainer4 = list_of_shap_values[3]

#### Calculating shap_values
Below we have used explainer to generate shape values for the test dataset using the shap_values() method of explainer. The explainer object has a base value to which it adds shape values for a particular sample in order to generate a final prediction. The base value is stored in the expected_value attribute of the explainer object. All model predictions will be generated by adding shap values generated for a particular sample to this expected value.

In [ ]:
# shap_values1_explainer1 = explainer1.shap_values(X_test[:1,:,:])

In [ ]:
# shap_values2_explainer1 = explainer1.shap_values(X_test)

In [ ]:
# shap_values1_explainer2 = explainer2.shap_values(X_test[:1,:,:])

In [ ]:
# shap_values2_explainer2 = explainer2.shap_values(X_test)

In [ ]:
# shap_values1_explainer3 = explainer3.shap_values(X_test[:1,:,:])

In [ ]:
# shap_values2_explainer3 = explainer3.shap_values(X_test)

In [ ]:
# shap_values1_explainer4 = explainer4.shap_values(X_test[:1,:,:])

In [ ]:
# shap_values2_explainer4 = explainer4.shap_values(X_test)

In [ ]:
# shap_values1_explainer5 = explainer5.shap_values(X_test[:1,:,:])

In [ ]:
# shap_values2_explainer5 = explainer5.shap_values(X_test)

In [ ]:
# shap_values1_explainer6 = explainer6.shap_values(X_test[:1,:,:])

In [ ]:
# shap_values2_explainer6 = explainer6.shap_values(X_test)

In [ ]:
# shap_values1_explainer7 = explainer7.shap_values(X_test[:1,:,:])

In [ ]:
# shap_values2_explainer7 = explainer7.shap_values(X_test)

In [ ]:
# shap_values1_explainer8 = explainer8.shap_values(X_test[:1,:,:])

In [ ]:
# shap_values2_explainer8 = explainer8.shap_values(X_test)

#### Creating pickle files to save the shap_values

In [ ]:
# with open("D:\Pedro\IST\ANO 2\SEM 2\Tese\gmestre\ICLR-RNN-CElegans\Experiment1\GRU\8\Singleoutput\Runy\pickles\shap_values1_explainer4.pkl", 'wb') as file:
#     pickle.dump(shap_values1_explainer4, file)

In [ ]:
# for i in range(3, 5):
#     for j in range(1, 3):
#         with open("D:\Pedro\IST\ANO 2\SEM 2\Tese\gmestre\ICLR-RNN-CElegans\Experiment1\GRU\8\Singleoutput\Runy\pickles\shap_values{}_explainer{}.pkl".format(j, i), 'wb') as file:
#             pickle.dump(eval("shap_values{}_explainer{}".format(j, i)), file)

In [ ]:
# for i in range(1, 9):
#     for j in range(1, 3):
#         with open("D:\Pedro\IST\ANO 2\SEM 2\Tese\gmestre\ICLR-RNN-CElegans\Experiment1\GRU\8\Singleoutput\Runy\pickles\shap_values{}_explainer{}.pkl".format(j, i), 'wb') as file:
#             pickle.dump(eval("shap_values{}_explainer{}".format(j, i)), file)

#### Print shap_values

In [ ]:
# print(shap_values1_explainer1)
# print(shap_values2_explainer1)

In [ ]:
# print(shap_values1_explainer2)
# print(shap_values2_explainer2)

In [ ]:
print(shap_values1_explainer3)
print(shap_values2_explainer3)

In [ ]:
print(shap_values1_explainer4)
print(shap_values2_explainer4)

In [ ]:
# print(shap_values1_explainer5)
# print(shap_values2_explainer5)

In [ ]:
# print(shap_values1_explainer6)
# print(shap_values2_explainer6)

In [ ]:
# print(shap_values1_explainer7)
# print(shap_values2_explainer7)

In [ ]:
# print(shap_values1_explainer8)
# print(shap_values2_explainer8)

In [ ]:
# print(np.array(shap_values1_explainer1).shape)
# print(np.array(shap_values2_explainer1).shape)

# print(np.array(shap_values1_explainer2).shape)
# print(np.array(shap_values2_explainer2).shape)

print(np.array(shap_values1_explainer3).shape)
print(np.array(shap_values2_explainer3).shape)

print(np.array(shap_values1_explainer4).shape)
print(np.array(shap_values2_explainer4).shape)

# print(np.array(shap_values1_explainer5).shape)
# print(np.array(shap_values2_explainer5).shape)

# print(np.array(shap_values1_explainer6).shape)
# print(np.array(shap_values2_explainer6).shape)

# print(np.array(shap_values1_explainer7).shape)
# print(np.array(shap_values2_explainer7).shape)

# print(np.array(shap_values1_explainer8).shape)
# print(np.array(shap_values2_explainer8).shape)

In [ ]:
shap_values1_explainer3_reshaped = np.reshape(np.array(shap_values1_explainer3), (1000, 1000, 4))

In [ ]:
shap_values1_explainer3_reshaped.shape

In [ ]:
shap_values1_explainer4_reshaped = np.reshape(np.array(shap_values1_explainer4), (1000, 1000, 4))

In [ ]:
shap_values1_explainer4_reshaped.shape

### expected_values

In [ ]:
# print(explainer1.expected_value)
# print(explainer2.expected_value)
print(explainer3.expected_value)
print(explainer4.expected_value)

In [ ]:
# print(explainer1.expected_value.shape)
# print(explainer2.expected_value.shape)
print(explainer3.expected_value.shape)
print(explainer4.expected_value.shape)

## Visualizations

### dependence_plots

In [ ]:
# code for X_test[:1,:,:]
shap.dependence_plot(
            ind=0,
            shap_values=shap_values1_explainer4_reshaped[0],
            features=X_test[:1,:,:][0],
            feature_names=nin_names,
            interaction_index=1)

In [ ]:
# code for X_test[:1,:,:]
shap.dependence_plot(
            ind=1,
            shap_values=shap_values1_explainer4_reshaped[0],
            features=X_test[:1,:,:][0],
            feature_names=nin_names,
            interaction_index=3)

In [ ]:
# code for X_test[:1,:,:]
for i in range(shap_values1_explainer4_reshaped.shape[0]):
    for j in range(len(nin_names)):
        print(i)
        print(j)
        shap.dependence_plot(
                ind=j,
                shap_values=shap_values1_explainer4_reshaped[i],
                features=X_test[0],
                feature_names=nin_names)

In [ ]:
# shap.dependence_plot(
#             ind=0,
#             shap_values=shap_values1_explainer4[0][0],
#             features=X_test[:1,:,:][0],
#             feature_names=nin_names,
#             interaction_index=1)

In [ ]:
# shap.dependence_plot(
#             ind=1,
#             shap_values=shap_values1_explainer4[0][0],
#             features=X_test[:1,:,:][0],
#             feature_names=nin_names,
#             interaction_index=3)

In [ ]:
# for i in range(shap_values1_explainer4.shape[0]):
#     for j in range(len(nin_names)):
#         shap.dependence_plot(
#                 ind=j,
#                 shap_values=shap_values1_explainer4[i][0],
#                 features=X_test[0],
#                 feature_names=nin_names)

#### Ensure the sum of Shapley values matches the difference between predicted value and expected value: For each instance in your dataset, the sum of the Shapley values across all features should add up to the difference between the model's predicted value for that instance and the expected value.

In [ ]:
np.array(shap_values1_explainer3).shape

In [ ]:
explainer3.expected_value.shape

In [ ]:
np.array(shap_values1_explainer4).shape

In [ ]:
explainer4.expected_value.shape

In [ ]:
predicted_values_1 = model.predict(X_test[:1,:,:])
predicted_values_1.shape

In [ ]:
predicted_values_10 = model.predict(X_test)
predicted_values_10.shape

In [ ]:
for i in range(len(X_test[:1,:,:])):
    for j in range(predicted_values_1.shape[1]):
        shap_sum = np.sum(shap_values1_explainer3[j][i])
        pred_diff = predicted_values_1[i, j] - explainer3.expected_value[j]
        print(f"Instance {i+1}, Output {j+1}: Shapley sum = \t{shap_sum}, \n\tPredicted difference = \t\t{pred_diff}")

In [ ]:
for i in range(len(X_test)):
    for j in range(predicted_values_10.shape[1]):
        shap_sum = np.sum(shap_values2_explainer3[j][i])
        pred_diff = predicted_values_10[i, j] - explainer3.expected_value[j]
        print(f"Instance {i+1}, Output {j+1}: Shapley sum = \t{shap_sum}, \n\tPredicted difference = \t\t{pred_diff}")

In [ ]:
for i in range(len(X_test[:1,:,:])):
    for j in range(predicted_values_1.shape[1]):
        shap_sum = np.sum(shap_values1_explainer4[j][i])
        pred_diff = predicted_values_1[i, j] - explainer4.expected_value[j]
        print(f"Instance {i+1}, Output {j+1}: Shapley sum = \t{shap_sum}, \n\tPredicted difference = \t\t{pred_diff}")

In [ ]:
for i in range(len(X_test)):
    for j in range(predicted_values_10.shape[1]):
        shap_sum = np.sum(shap_values2_explainer4[j][i])
        pred_diff = predicted_values_10[i, j] - explainer4.expected_value[j]
        print(f"Instance {i+1}, Output {j+1}: Shapley sum = \t{shap_sum}, \n\tPredicted difference = \t\t{pred_diff}")

In [ ]:
def verify_shap_values(shap_values, expected_values, predicted_values):
    """
    Verify the correctness of SHAP values.

    Args:
        shap_values (numpy.ndarray): Array of SHAP values with shape (n_instances, 1, n_features, n_classes).
        expected_values (numpy.ndarray): Array of expected values with shape (n_instances,).
        predicted_values (numpy.ndarray): Array of predicted values with shape (1, n_instances).

    Returns:
        bool: True if the SHAP values are correctly calculated, False otherwise.
    """
    
    # Step 1: Verify shapes
    n_instances, _, n_features, n_classes = shap_values.shape
    _, n_instances_predicted = predicted_values.shape

    if n_instances != n_instances_predicted:
        print("Error: Number of instances in shap_values and predicted_values do not match.")
        return False

    # Step 2: Check the sum property
    for i in range(n_instances):
        sum_shap_values = np.sum(shap_values[i, 0, :, :])
        diff_predicted_expected = predicted_values[0, i] - expected_values[i]
        
        print(sum_shap_values)
        print(diff_predicted_expected)

        if not np.isclose(sum_shap_values, diff_predicted_expected):
            print(f"Error: Sum of SHAP values for instance {i} does not match the difference between predicted and expected values.")
            return False

    return True

In [ ]:
verify_shap_values(np.array(shap_values1_explainer3), explainer3.expected_value, predicted_values_1)

In [ ]:
verify_shap_values(np.array(shap_values2_explainer3), explainer3.expected_value, predicted_values_10)

In [ ]:
verify_shap_values(np.array(shap_values1_explainer4), explainer4.expected_value, predicted_values_1)

In [ ]:
verify_shap_values(np.array(shap_values2_explainer4), explainer4.expected_value, predicted_values_10)